This is a final project for the course STA410. The project aims to perform anomaly detection within simple system log events using Hidden Markov Models, Log-Likelihoods, and Z-scores.

In [125]:
import scipy as sc
import numpy as np
import random
!pip install simpy
import simpy as sp
!pip install hmmlearn
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
import statistics

## Simulating the System User Log Events Sequences using SimPy

In [126]:
NORMAL_EVENTS = ["file_access", "process_start", "network_request",
    "print_document", "open_application", "save_file", "email_sent", "calendar_event_created",
    "drive_mount", "clipboard_copy", "clipboard_paste", "settings_viewed", "folder_created",
    "tab_switched", "browser_opened", "form_submitted", "search_query", "notification_dismissed",
    "screen_locked", "screen_unlocked", "update_checked", "vpn_connected", "usb_connected",
    "idle_timeout", "desktop_icon_clicked", "screenshot_taken", "password_entered", "window_minimized", "password_changed", "permission_request"]

ABNORMAL_EVENTS = ["unauthorized_file_access", "inappropriate_searches", "data_exfiltration", "access_restricted_area",
    "usb_file_transfer", "disabled_antivirus", "multiple_failed_logins", "download_torrent",
    "privilege_escalation", "disabled_firewall", "remote_desktop_started",
    "shadow_copy_deleted", "unauthorized_process", "unusual_data_volume",
    "external_backup_created", "suspicious_registry_edit", "hidden_file_created", "encoded_email_sent",
    "executable_downloaded", "log_cleared", "system_time_changed", "ransomware_alert",
    "suspicious_macro_execution", "encrypted_container_mounted", "unauthorized_command_prompt",
    "malicious_script_executed", "rootkit_detected", "browser_exploit_triggered", "fake_cert_installed", "unknown_device_connected"]

USERS = ["dign6", "aliv9", "vinc1", "manj7","ritd0", "tond3", "gana4", "gian2", "shaw5", "lore8", "mexa4", "zoti3", "japu1", "vilo8", "dare6", "soqi9", "teza0", "boku7", "rima5", "nely2",
    "waro6", "xipa0", "guto3", "cenu9", "lira1", "pemo8", "hika7", "dofu4", "moxa2", "yave5"]

class System:

    """
    ------ Attributes ------
    env: SimPy Environment
    (int) shift_end: The time when the shift ends.
    (list) log: List of logged system events. Events are in a tuple of (timestamp, event).
    (list) users: Registered users.
    (dict) user_activity: Users and their log events at the time performed.

    """

    def __init__(self, env, user_ids, shift_end_time):
        self.env = env
        self.shift_end = env.timeout(shift_end_time)
        self.log = []
        self.users = user_ids
        self.user_activity = {}

        random.shuffle(self.users)

        for user_id in self.users:
            env.process(self.user_session(user_id))

    def record_user_activity(self, user_id, activity, timestamp):
        """
        Records the user activity.
        """
        if user_id not in self.user_activity:
            self.user_activity[user_id] = [f"{activity} at time {timestamp}"]
        else:
            self.user_activity[user_id].append(f"{activity} at time {timestamp}")

    def user_session(self, user_id):
        """
        Entire session of a user. Randomly choose an amount of actions to take, and perform that amount of the normal events.

        """
        timestamp = self.env.now

        self.log_event(timestamp, f"User {user_id} logged in", user_id)

        # simulate random user activity
        for _ in range(random.randint(10, 30)):  # Perform 10-30 actions
            if random.random() < 0.20: # 20% chance user will do an anomalous activity
                yield self.env.timeout(random.randint(2,5)) # pause the simulation and function for a random time between 2-5
                event = random.choice(ABNORMAL_EVENTS)
                timestamp = self.env.now
                self.log_event(timestamp, f"User {user_id} {event}", user_id)

            yield self.env.timeout(random.randint(1,3)) # pause the simulation and function for a random time between 1-3
            event = random.choice(NORMAL_EVENTS)
            timestamp = self.env.now
            self.log_event(timestamp, f"User {user_id} {event}", user_id)

        yield self.shift_end # user sessions will stop once their shift is over and they log out
        timestamp = self.env.now
        self.log_event(timestamp, f"User {user_id} logged out", user_id)

    def log_event(self, timestamp, event, user_id):
        """
        Logs a system event.
        """
        self.log.append((timestamp, event))
        self.record_user_activity(user_id, event, timestamp)


## Run the Simulation

In [127]:
# set up the simulation
WORK_TIME = 40
env = sp.Environment()
system = System(env, USERS, WORK_TIME)

env.run() # runs the simulation

for user in system.user_activity: # lists what each user did
    print(f"{user}:")
    for event in system.user_activity[user]:
        print(f" - {event}")

moxa2:
 - User moxa2 logged in at time 0
 - User moxa2 search_query at time 3
 - User moxa2 drive_mount at time 5
 - User moxa2 search_query at time 6
 - User moxa2 vpn_connected at time 8
 - User moxa2 screen_unlocked at time 10
 - User moxa2 tab_switched at time 13
 - User moxa2 screen_locked at time 14
 - User moxa2 notification_dismissed at time 15
 - User moxa2 desktop_icon_clicked at time 17
 - User moxa2 desktop_icon_clicked at time 18
 - User moxa2 vpn_connected at time 20
 - User moxa2 screenshot_taken at time 23
 - User moxa2 inappropriate_searches at time 28
 - User moxa2 screen_unlocked at time 29
 - User moxa2 password_entered at time 30
 - User moxa2 clipboard_paste at time 31
 - User moxa2 unauthorized_process at time 33
 - User moxa2 print_document at time 36
 - User moxa2 clipboard_paste at time 37
 - User moxa2 drive_mount at time 39
 - User moxa2 folder_created at time 40
 - User moxa2 drive_mount at time 43
 - User moxa2 vpn_connected at time 44
 - User moxa2 disabl

##  Train HMM on Normal Log Sequences


In [128]:
# convert system events to numbers for HMM to learn

all_events = NORMAL_EVENTS + ABNORMAL_EVENTS

encoder = LabelEncoder()
encoder.fit(all_events)

user_sequences = defaultdict(list)

for timestamp, raw_event in system.log:
    parts = raw_event.split(" ", 2)
    if len(parts) == 3:
        _, user, event = parts
        if event in all_events:
            user_sequences[user].append(event)

# want to train HMM on only normal sequences

normal_user_sequences = {
    user: [event for event in seq if event in NORMAL_EVENTS] for user, seq in user_sequences.items()
}

encoded_normal_sequences = {
    user: encoder.transform(seq) for user, seq in normal_user_sequences.items()
}

# train HMM on the normal sequences

X = np.concatenate([np.array(seq).reshape(-1, 1) for seq in encoded_normal_sequences.values()])

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

lengths = [len(seq) for seq in encoded_normal_sequences.values()]
total_samples = sum(lengths)

model = GaussianHMM(n_components=5)
model.fit(X_normalized, lengths=lengths)


GaussianHMM(n_components=5)

## Scoring New Sequences Using Log-Likelihood

In [132]:
# get all user scores to compare

all_encoded_sequences = {
    user: encoder.transform(seq) for user, seq in user_sequences.items()
}

all_X = np.concatenate([np.array(seq).reshape(-1, 1) for seq in all_encoded_sequences.values()])
all_X_normalized = scaler.transform(all_X)

# score users using log-likelihood, a higher (less negative) score means the user behaviour looks normal, and much lower (very negative) score means it is abnormal

user_scores = {}
index = 0
for user, seq in all_encoded_sequences.items():
    seq_normalized = all_X_normalized[index:index+len(seq)].reshape(-1, 1)
    user_scores[user] = model.score(seq_normalized)
    index += len(seq)

## Detect Anomalies Using Z-scores

In [133]:
# use z-scores to compare how unusual one sequence of events may be to the rest

scores = list(user_scores.values())
mean = statistics.mean(scores)
std = statistics.stdev(scores)

for user, score in user_scores.items():
    z = (score - mean) / std
    print(f"{user}: log-likelihood = {score:.2f}, z = {z:.2f}")
    if z < -1:
        print("SUSPICIOUS ACTIVITY DETECTED")

guto3: log-likelihood = -20.92, z = 1.21
vilo8: log-likelihood = -34.14, z = -0.15
dign6: log-likelihood = -39.07, z = -0.65
manj7: log-likelihood = -37.93, z = -0.54
gana4: log-likelihood = -42.34, z = -0.99
yave5: log-likelihood = -19.95, z = 1.31
zoti3: log-likelihood = -31.89, z = 0.08
aliv9: log-likelihood = -17.65, z = 1.55
soqi9: log-likelihood = -19.77, z = 1.33
rima5: log-likelihood = -31.16, z = 0.16
dofu4: log-likelihood = -19.90, z = 1.32
cenu9: log-likelihood = -52.85, z = -2.07
SUSPICIOUS ACTIVITY DETECTED
hika7: log-likelihood = -47.38, z = -1.51
SUSPICIOUS ACTIVITY DETECTED
japu1: log-likelihood = -47.63, z = -1.53
SUSPICIOUS ACTIVITY DETECTED
shaw5: log-likelihood = -36.45, z = -0.38
lore8: log-likelihood = -23.76, z = 0.92
moxa2: log-likelihood = -39.09, z = -0.66
lira1: log-likelihood = -26.66, z = 0.62
vinc1: log-likelihood = -30.31, z = 0.25
nely2: log-likelihood = -38.99, z = -0.64
boku7: log-likelihood = -39.65, z = -0.71
dare6: log-likelihood = -18.08, z = 1.50
